In [100]:
from math import sqrt,acos,pi
from decimal import Decimal,getcontext

getcontext().prec=30

class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(x) for x in coordinates])
            self.dimension = len(coordinates)
            
        except ValueError:
            raise ValueError("The coordinates must be nonempty")
            
        except TypeError:
            raise TypeError("The coordinates must be an iterable")
            
    def __str__(self):
        return "Vector: {}".format(self.coordinates)
    
    def __eq__(self,v):
        return self.coordinates == v.coordinates
            
    def plus (self,v):
        new_coordinates=[x+y for x,y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)

    def minus (self,v):
        new_coordinates=[x-y for x,y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)

    def times_scalar(self,c):
        new_coordinates=[x*Decimal(c) for x in self.coordinates]
        return Vector(new_coordinates)
    
    def magnitude(self):
        coordinates_squared=[x**2 for x in self.coordinates]
        return Decimal(sqrt(sum(coordinates_squared)))
    
    def normalized(self):
        try:
            magnitude=self.magnitude()
            return self.times_scalar(Decimal("1.0")/magnitude)
        except ZeroDivisionError:
            raise Exception ("can't normalize the zero vector")
    
    def dot(self,v):
        return sum([x*y for x,y in zip(self.coordinates, v.coordinates)])
    
    def angle_with(self,v,in_degrees=False):
        try:
            u1=self.normalized()
            u2=v.normalized()
            angle_in_radians = acos(u1.dot(u2))
            
            if in_degrees:
                degrees_per_radian=180./pi
                return angle_in_radians*degrees_per_radian
            else:
                return angle_in_radians   
        except Exception as e:
            if str(e)==self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception("cannot compute an angle with the zero vector")
            else: raise e
                
    def is_orthogonal_to(self, v, tolerance=1e-10):
        return abs(self.dot(v)) < tolerance
    
    def is_parallel_to(self,v):
        return (self.is_zero() or v.is_zero() or
                 self.angle_with(v)==0 or self.angle_with(v)==pi)
    
    def is_zero(self, tolerance=1e-10):
        return self.magnitude()<tolerance
    
    def component_parallel_to(self,basis):
        try:
            u=basis.normalized()
            weight=self.dot(u)
            return u.times_scalar(weight)
              
        except Exception as e:
            if str(e)==self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else: raise e
                
    def component_orthogonal_to(self,basis):
        try:
            projection=self.component_parallel_to(basis)
            return self.minus(projection)
              
        except Exception as e:
            if str(e)==self.NO_UNIQUE_PARALLEL_COMPONENT_MSG:
                raise Exception(self.NO_UNIQUE_ORTHOGONAL_COMPONENT_MSG)
            else: raise e
    
    def cross(self,v):
        try:
            x_1,y_1,z_1=self.coordinates
            x_2,y_2,z_2=v.coordinates
            new_coordinates=[y_1*z_2-y_2*z_1,
                             -(x_1*z_2-x_2*z_1),
                             x_1*y_2-x_2*y_1]
            return Vector(new_coordinates)
        except ValueError as e:
            msg=str(e)
            if msg=="need more than 2 values to unpack":
                self_embedded_in_R3=Vector(self.coordinates+("0",))
                v_embedded_in_R3=Vector(v.coordinates+("0",))
                return self_embedded_in_R3.cross(v_embedded_in_R3)
            elif (msg=="too many values to unpack" or msg=="need more than 1 value to unpack"):
                raise Exception (self.ONLY_DEFINED_IN_TWO_THREE_DIMS_MSG)
            else: raise e
                
    def area_of_parallelogram_with(self,v):
        cross_product=self.cross(v)
        return cross_product.magnitude()
    
    def area_of_triangle_with(self,v):
        return self.area_of_parallelogram_with(v)/Decimal("2.0")

In [53]:
sample= zip([1,2,3],[2,3])

In [55]:
sample

In [101]:
my_vector=Vector([1,2,3])
w=Vector([1,-2,4])


In [102]:
print (my_vector.area_of_parallelogram_with(w))

14.5945195193264236621644158731214702129364013671875


In [87]:
print (my_vector.normalized())

Vector: (Decimal('0.267261241912424388690952378529'), Decimal('0.534522483824848777381904757058'), Decimal('0.801783725737273166072857135587'))


In [88]:
print (my_vector.times_scalar(2))

Vector: (Decimal('2'), Decimal('4'), Decimal('6'))


In [93]:
print (my_vector.angle_with(w,in_degrees=True))

58.33911722540479


In [2]:
from decimal import Decimal, getcontext

getcontext().prec = 30


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps